# Running MD simulations using nff and ASE

This Jupyter Notebook shows how the `nff` package interfaces with the Atomistic Simulation Environment (ASE). We assume the user went through tutorial `01_training`, so we can load the pretrained models without having to train them again.

As before, importing the dependencies:

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import torch
from ase import Atoms
from ase.md.verlet import VelocityVerlet

from nff.io import NeuralFF, AtomsBatch
from nff.md.nve import Dynamics
from nff.data import Dataset
from nff.train import load_model, evaluate
import nff.utils.constants as const
from ase import units

## Loading the relevant data

We reload the dataset and create a `GraphLoader` as we did last time:

In [2]:
dataset = Dataset.from_file('data/dataset.pth.tar')

### Creating Atoms

As before, we can create an `Atoms` object from any element of the dataset. Let's take the first one, for simplicity:

In [3]:
props = dataset[0].copy()
atoms = AtomsBatch(positions=props['nxyz'][:, 1:], numbers=props['nxyz'][:, 0], props=props)

### Creating the ASE calculator

Now we just have to load the ASE calculator from a pretrained model. One way of doing so is through the in-build `from_file` method. You just have to specify the folder where the model was trained and subsequently stored.

In [4]:
nff_ase = NeuralFF.from_file('sandbox/', device=3)

Assigning this calculator to `atoms` is easy:

In [5]:
atoms.set_calculator(nff_ase)

### Configuring the dynamics for the system

In this example, we will run an NVE dynamics simulation. We will use the default parameters there implemented to run a trajectory for an ethanol molecule. The parameters we will specify are the following:

* `T_init`: initial temperature of the simulation
* `time_step`: time step in femtoseconds
* `thermostat`: ASE integrator to use when performing the simulation
* `thermostat_params`: keyword arguments for ase.Integrator class, will be different case-by-case
* `steps`: number of steps to simulate
* `save_frequency`: how often (in steps) save the pose of the molecule in a file
* `nbr_list_update_freq`: how often (in steps) to update the neighbor list (not yet implemented)
* `thermo_filename`: output file for the thermodynamics log
* `traj_filename`: output file for the ASE trajectory file
* `skip`: number of initial frames to skip when recording the trajectory

In [6]:
md_params = {
    'T_init': 450,
    'time_step': 0.5,
#     'thermostat': NoseHoover,   # or Langevin or NPT or NVT or Thermodynamic Integration
#     'thermostat_params': {'timestep': 0.5 * units.fs, "temperature": 120.0 * units.kB,  "ttime": 20.0}
    'thermostat': VelocityVerlet,  
    'thermostat_params': {'timestep': 0.5 * units.fs},
    'steps': 200,
    'save_frequency': 10,
    'nbr_list_update_freq': 20,
    'thermo_filename': 'thermo.log',
    'traj_filename': 'atoms.traj',
    'skip': 0
}

In [7]:
nve = Dynamics(atoms, md_params)
nve.run()

Time[ps]      Etot[eV]     Epot[eV]     Ekin[eV]    T[K]
0.0000           1.7715       1.3434       0.4282   368.0

0.0050           1.7721       1.4227       0.3495   300.4

0.0100           1.7718       1.3977       0.3740   321.5

0.0150           1.7720       1.4554       0.3166   272.1

0.0200           1.7714       1.3960       0.3755   322.7

0.0250           1.7719       1.4456       0.3263   280.5

0.0300           1.7721       1.4628       0.3093   265.8

0.0350           1.7719       1.4207       0.3512   301.9

0.0400           1.7717       1.3760       0.3958   340.2

0.0450           1.7715       1.3262       0.4453   382.7

0.0500           1.7721       1.4202       0.3519   302.5

0.0550           1.7718       1.3722       0.3996   343.5

0.0600           1.7718       1.4053       0.3664   315.0

0.0650           1.7716       1.4480       0.3237   278.2

0.0700           1.7721       1.5253       0.2468   212.1

0.0750           1.7719       1.4752       0.2967   255.0


The dynamics conserved the energy. The temperature varied throughout the simulation, as expected.

## Visualizing the trajectory

To visualize the trajectory in this Jupyter Notebook, you will have to install the package [nglview](https://github.com/arose/nglview).

In [9]:
import nglview as nv
from ase.io import Trajectory

_ColormakerRegistry()

Displaying the trajectory:

In [10]:
%matplotlib notebook

traj = Trajectory('atoms.traj')
nv.show_asetraj(traj)

NGLWidget(max_frame=20)

Looks like the atoms are still together. Visual inspection says that the trajectory is reasonable. Yay for `nff`!